In [33]:
import pandas as pd
import numpy as np

# 데이터 전처리
## 법정동 데이터

In [34]:
# 법정동 코드와 이름이 있는 파일을 로드
sgg_bjd_df = pd.read_excel('data/원본/법정동코드 조회자료.xls')
sgg_bjd_df.head()

,법정동코드,법정동명,폐지구분
0,1100000000,서울특별시,현존
1,1111000000,서울특별시 종로구,현존
2,1111010100,서울특별시 종로구 청운동,현존
3,1111010200,서울특별시 종로구 신교동,현존
4,1111010300,서울특별시 종로구 궁정동,현존


In [35]:
# 편의를 위해 행이름 변경
sgg_bjd_df.columns = ['전체코드', '법정동명(전체)', '폐지구분']

In [36]:
sgg_bjd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   전체코드      1112 non-null   int64 
 1   법정동명(전체)  1112 non-null   object
 2   폐지구분      1112 non-null   object
dtypes: int64(1), object(2)
memory usage: 26.2+ KB


### 1 법정동코드 분리
법정동 전체 코드(10자리)는 시군구코드(5자리)와 법정동코드(5자리)가 합쳐져있는 형태입니다. 이를 먼저 분리해줍니다.

In [37]:
# 법정동코드를 시군구와 법정동으로 분리
# 법정동코드 = 시군구(5자리)+법정동(5자리)
def splitCode(row):
    stringCode = str(row['전체코드'])
    row['시군구코드'] = int(stringCode[:5])
    row['법정동코드'] = int(stringCode[5:])
    return row

sgg_bjd_df = sgg_bjd_df.apply(splitCode, axis=1)
sgg_bjd_df


,전체코드,법정동명(전체),폐지구분,시군구코드,법정동코드
0,1100000000,서울특별시,현존,11000,0
1,1111000000,서울특별시 종로구,현존,11110,0
2,1111010100,서울특별시 종로구 청운동,현존,11110,10100
3,1111010200,서울특별시 종로구 신교동,현존,11110,10200
4,1111010300,서울특별시 종로구 궁정동,현존,11110,10300
...,...,...,...,...,...
1107,1174010600,서울특별시 강동구 둔촌동,현존,11740,10600
1108,1174010700,서울특별시 강동구 암사동,현존,11740,10700
1109,1174010800,서울특별시 강동구 성내동,현존,11740,10800
1110,1174010900,서울특별시 강동구 천호동,현존,11740,10900


### 2. 법정동 코드가 0인 데이터 제거
법정동 코드가 0이라는 것은 법정동 이름이 기재가 안되어있다는 의미입니다. 이런 데이터들은 제거합니다

In [38]:
sgg_bjd_df = sgg_bjd_df[sgg_bjd_df['법정동코드']!=0]
sgg_bjd_df.sample(5)

,전체코드,법정동명(전체),폐지구분,시군구코드,법정동코드
826,1156001600,서울특별시 영등포구 오쇠동,폐지,11560,1600
414,1126090900,서울특별시 중랑구 상봉2동,폐지,11260,90900
1000,1165091100,서울특별시 서초구 방배3동,폐지,11650,91100
19,1111011800,서울특별시 종로구 내수동,현존,11110,11800
638,1141010400,서울특별시 서대문구 미근동,현존,11410,10400


### 3. 법정동 전체명을 신군구명과 법정동명으로 분리

In [39]:
def splitName(row):
    add = row['법정동명(전체)'].split(' ')
    row['시군구명'] = add[1]
    row['법정동명'] = add[2]
    return row

sgg_bjd_df = sgg_bjd_df.apply(splitName, axis=1)

### 4. 필요없는 행 삭제 및 저장

In [40]:
sgg_bjd_df.head()

,전체코드,법정동명(전체),폐지구분,시군구코드,법정동코드,시군구명,법정동명
2,1111010100,서울특별시 종로구 청운동,현존,11110,10100,종로구,청운동
3,1111010200,서울특별시 종로구 신교동,현존,11110,10200,종로구,신교동
4,1111010300,서울특별시 종로구 궁정동,현존,11110,10300,종로구,궁정동
5,1111010400,서울특별시 종로구 효자동,현존,11110,10400,종로구,효자동
6,1111010500,서울특별시 종로구 창성동,현존,11110,10500,종로구,창성동


In [41]:
# 필요한 행만 추출
sgg_bjd_df = sgg_bjd_df[['전체코드', '시군구코드', '시군구명', '법정동코드', '법정동명', '폐지구분']].reset_index(drop=True)
sgg_bjd_df.head()

,전체코드,시군구코드,시군구명,법정동코드,법정동명,폐지구분
0,1111010100,11110,종로구,10100,청운동,현존
1,1111010200,11110,종로구,10200,신교동,현존
2,1111010300,11110,종로구,10300,궁정동,현존
3,1111010400,11110,종로구,10400,효자동,현존
4,1111010500,11110,종로구,10500,창성동,현존


In [42]:
# 저장하기 (csv)
sgg_bjd_df.to_csv('data/전처리/시군구_법정동_코드.csv', index=False)

## 전력량데이터

In [43]:
# 전력량 데이터 가져오기
power_df = pd.read_csv('data/원본/법정동별시간별전력사용량.csv')
power_df.head()

,SIGUNGU_CD,BJDONG_CD,USE_YM,USE_HM,FDRCT_VLD_KWH
0,11650,10700,20220628,100,10782.0565
1,11650,10800,20220628,100,11394.8635
2,11650,10900,20220628,100,7273.9620
3,11740,10300,20220628,100,11008.8110
4,11710,11300,20220628,100,2905.1120


In [44]:
power_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9754804 entries, 0 to 9754803
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   SIGUNGU_CD     int64  
 1   BJDONG_CD      int64  
 2   USE_YM         int64  
 3   USE_HM         int64  
 4   FDRCT_VLD_KWH  float64
dtypes: float64(1), int64(4)
memory usage: 372.1 MB


In [45]:
# column명이 너무 길어서 column 재정의하기, 시간을 1~24로 변경
power_df.columns = ['시군구코드', '법정동코드', '날짜', '시', '파워']
power_df['시'] = power_df['시']/100
power_df['시'] = power_df['시'].astype(np.int16)
power_df.head()

,시군구코드,법정동코드,날짜,시,파워
0,11650,10700,20220628,1,10782.0565
1,11650,10800,20220628,1,11394.8635
2,11650,10900,20220628,1,7273.9620
3,11740,10300,20220628,1,11008.8110
4,11710,11300,20220628,1,2905.1120


### 1. 중복 데이터 삭제

In [46]:
power_df[(power_df['법정동코드']==10300) & (power_df['날짜']==20230811) & (power_df['시군구코드']==11110)]

,시군구코드,법정동코드,날짜,시,파워
4126106,11110,10300,20230811,1,26.568
4126182,11110,10300,20230811,1,26.568
4126216,11110,10300,20230811,1,26.568
4126331,11110,10300,20230811,1,26.568
4127972,11110,10300,20230811,2,24.590
...,...,...,...,...,...
4165822,11110,10300,20230811,23,24.458
4168976,11110,10300,20230811,24,24.999
4169061,11110,10300,20230811,24,24.999
4169092,11110,10300,20230811,24,24.999


위의 데이터를 보면 같은 지역(시군구코드+법정동코드)과 같은 시간대의 동일한 데이터가 여럿 있는것이 보입니다.
이런 중복 데이터들을 전부 하나의 데이터로 변경해야합니다.

In [47]:
power_drop_dup = power_df.drop_duplicates().reset_index(drop=True)
power_drop_dup

,시군구코드,법정동코드,날짜,시,파워
0,11650,10700,20220628,1,10782.0565
1,11650,10800,20220628,1,11394.8635
2,11650,10900,20220628,1,7273.9620
3,11740,10300,20220628,1,11008.8110
4,11710,11300,20220628,1,2905.1120
...,...,...,...,...,...
4348555,11110,16800,20240128,24,2497.7400
4348556,11110,16900,20240128,24,1473.3340
4348557,11110,17100,20240128,24,775.8270
4348558,11110,16700,20240128,24,738.6430


In [51]:
power_df.shape, power_drop_dup.shape

((9754804, 5), (4348560, 5))

In [50]:
power_drop_dup.to_csv('data/전처리/전력데이터_중복제거.csv', index=False)

### 2. 데이터 보간
중복 데이터도 있지만 24시간 전부 기록되지 않은 데이터들 역시 존재합니다. 데이터가 연속성을 띄우기에 선형 보간을 이용하여 중간중간 띄어져있는 데이터를 채웁니다

In [ ]:
# 하루 데이터가 24개 이하인 데이터 찾아보기 (총 1207의 법정동 날짜가 하루 24개 이하의 데이터가 존재)
count = power_drop_dup[['시군구코드', '법정동코드', '날짜']].value_counts()
count[count !=24]


시군구코드  법정동코드  날짜      
11410  10300  20240120    23
11110  12500  20231214    23
              20230220    23
              20230218    23
11170  13200  20230116    23
                          ..
11140  11600  20240120     9
11110  15000  20230102     9
11140  15600  20240120     7
11110  15000  20230101     3
              20230604     1
Length: 1207, dtype: int64

In [ ]:
day_group = power_drop_dup.groupby(by=['시군구코드', '법정동코드', '날짜'])
result_list = []
for (sgg, bjd, date), group in day_group:
    all_hours = range(1,25)
    dataframe = pd.DataFrame({'시군구코드':sgg, '법정동코드':bjd, '날짜':date, '시':all_hours})
    merged = dataframe.merge(group, on=['시군구코드', '법정동코드', '날짜', '시'], how='left')
    
    # 선형 비례하는 방식으로 결측값 보간
    merged['파워'] = merged['파워'].interpolate(method='linear')
    merged['파워'] = merged['파워'].fillna(method='ffill').fillna(method='bfill')
    result_list.append(merged)

power_interpolate_df = pd.concat(result_list, ignore_index=True)
power_interpolate_df = power_interpolate_df.sort_values(['시군구코드', '법정동코드', '날짜', '시']).reset_index(drop=True)
power_interpolate_df.sample()

,시군구코드,법정동코드,날짜,시,파워
3316709,11530,11000,20230606,6,847.1805


In [ ]:
# 결측값 보간 이후 하루 데이터가 24개 이하인 데이터
count = power_interpolate_df[['시군구코드', '법정동코드', '날짜']].value_counts()
count[count !=24]

Series([], dtype: int64)

In [ ]:
# 결측치 수
power_interpolate_df.isnull().sum()

시군구코드    0
법정동코드    0
날짜       0
시        0
파워       0
dtype: int64

In [ ]:
# 결측값 보간 데이터 저장
power_interpolate_df.to_csv('data/전처리/전력데이터_결측값보간.csv', index=False)

### 3. 시간별 데이터를 일별 사용량 데이터로 변경

In [ ]:
power_day_df = power_interpolate_df.groupby(by=['시군구코드', '법정동코드', '날짜']).sum().reset_index()
power_day_df = power_day_df.drop(columns='시')
power_day_df

,시군구코드,법정동코드,날짜,파워
0,11110,10100,20221204,146294.6135
1,11110,10100,20221205,175633.8270
2,11110,10100,20221206,156084.1910
3,11110,10100,20221208,177018.8420
4,11110,10100,20221209,158467.1690
...,...,...,...,...
181403,11740,11000,20240124,115124.5470
181404,11740,11000,20240125,99907.5000
181405,11740,11000,20240126,94818.5120
181406,11740,11000,20240127,86792.8280


In [ ]:
#저장하기
power_day_df.to_csv('data/전처리/전력데이터_일별사용량.csv', index=False)

### 4. 전력데이터와 법정명 코드데이터 합치기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 합칠 데이터 읽어오기
power_day_df = pd.read_csv('data/전처리/전력데이터_일별사용량.csv')
sgg_bjd_df = pd.read_csv('data/전처리/시군구_법정동_코드.csv')

In [ ]:
power_day_df.head()

,시군구코드,법정동코드,날짜,파워
0,11110,10100,20221204,146294.6135
1,11110,10100,20221205,175633.8270
2,11110,10100,20221206,156084.1910
3,11110,10100,20221208,177018.8420
4,11110,10100,20221209,158467.1690


In [ ]:
sgg_bjd_df.head()

,전체코드,시군구코드,시군구명,법정동코드,법정동명,폐지구분
0,1111010100,11110,종로구,10100,청운동,현존
1,1111010200,11110,종로구,10200,신교동,현존
2,1111010300,11110,종로구,10300,궁정동,현존
3,1111010400,11110,종로구,10400,효자동,현존
4,1111010500,11110,종로구,10500,창성동,현존


In [ ]:
power_day_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181408 entries, 0 to 181407
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   시군구코드   181408 non-null  int64  
 1   법정동코드   181408 non-null  int64  
 2   날짜      181408 non-null  int64  
 3   파워      181408 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 5.5 MB


In [ ]:
sgg_bjd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086 entries, 0 to 1085
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   전체코드    1086 non-null   int64 
 1   시군구코드   1086 non-null   int64 
 2   시군구명    1086 non-null   object
 3   법정동코드   1086 non-null   int64 
 4   법정동명    1086 non-null   object
 5   폐지구분    1086 non-null   object
dtypes: int64(3), object(3)
memory usage: 51.0+ KB


In [ ]:
merged = power_day_df.merge(sgg_bjd_df, on=['시군구코드', '법정동코드'], how='left')
merged

,시군구코드,법정동코드,날짜,파워,전체코드,시군구명,법정동명,폐지구분
0,11110,10100,20221204,146294.6135,1111010100,종로구,청운동,현존
1,11110,10100,20221205,175633.8270,1111010100,종로구,청운동,현존
2,11110,10100,20221206,156084.1910,1111010100,종로구,청운동,현존
3,11110,10100,20221208,177018.8420,1111010100,종로구,청운동,현존
4,11110,10100,20221209,158467.1690,1111010100,종로구,청운동,현존
...,...,...,...,...,...,...,...,...
181403,11740,11000,20240124,115124.5470,1174011000,강동구,강일동,현존
181404,11740,11000,20240125,99907.5000,1174011000,강동구,강일동,현존
181405,11740,11000,20240126,94818.5120,1174011000,강동구,강일동,현존
181406,11740,11000,20240127,86792.8280,1174011000,강동구,강일동,현존


In [ ]:
# 폐지구분이 폐지인 데이터
merged[merged['폐지구분']=='폐지']

,시군구코드,법정동코드,날짜,파워,전체코드,시군구명,법정동명,폐지구분


In [ ]:
# 폐지구분이 폐지인 데이터가 없기에 폐지구분을 제거
merged = merged.drop(columns=['폐지구분'])
merged.head()

,시군구코드,법정동코드,날짜,파워,전체코드,시군구명,법정동명
0,11110,10100,20221204,146294.6135,1111010100,종로구,청운동
1,11110,10100,20221205,175633.8270,1111010100,종로구,청운동
2,11110,10100,20221206,156084.1910,1111010100,종로구,청운동
3,11110,10100,20221208,177018.8420,1111010100,종로구,청운동
4,11110,10100,20221209,158467.1690,1111010100,종로구,청운동


In [ ]:
# 행을 재배치하여 더 알아보기 쉽게하기
cols = ['전체코드', '시군구코드', '시군구명', '법정동코드', '법정동명', '날짜', '파워']
merged = merged[cols]
merged.head()

,전체코드,시군구코드,시군구명,법정동코드,법정동명,날짜,파워
0,1111010100,11110,종로구,10100,청운동,20221204,146294.6135
1,1111010100,11110,종로구,10100,청운동,20221205,175633.8270
2,1111010100,11110,종로구,10100,청운동,20221206,156084.1910
3,1111010100,11110,종로구,10100,청운동,20221208,177018.8420
4,1111010100,11110,종로구,10100,청운동,20221209,158467.1690


### 5. 날짜 데이터를 datetime 형식으로

In [ ]:
merged['날짜'] = pd.to_datetime(merged['날짜'].astype(str))
merged

,전체코드,시군구코드,시군구명,법정동코드,법정동명,날짜,파워
0,1111010100,11110,종로구,10100,청운동,2022-12-04,146294.6135
1,1111010100,11110,종로구,10100,청운동,2022-12-05,175633.8270
2,1111010100,11110,종로구,10100,청운동,2022-12-06,156084.1910
3,1111010100,11110,종로구,10100,청운동,2022-12-08,177018.8420
4,1111010100,11110,종로구,10100,청운동,2022-12-09,158467.1690
...,...,...,...,...,...,...,...
181403,1174011000,11740,강동구,11000,강일동,2024-01-24,115124.5470
181404,1174011000,11740,강동구,11000,강일동,2024-01-25,99907.5000
181405,1174011000,11740,강동구,11000,강일동,2024-01-26,94818.5120
181406,1174011000,11740,강동구,11000,강일동,2024-01-27,86792.8280


In [ ]:
# 저장하기
merged.to_csv('data/전처리/법정동_일별_전력량.csv', index=False)

## 날씨 데이터

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
weather_df = pd.read_csv('data/원본/날씨데이터/일별/SURFACE_ASOS_108_DAY_2023_2023_2024.csv', encoding='cp949')
weather_df.head()

,지점,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),10분 최다강수량 시각(hhmi),...,평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr)
0,108,2023-01-01,-0.2,-4.3,2350,3.8,1343,NaN,NaN,NaN,...,0.6,2.8,6.3,10.1,15.9,18.0,1.6,2.3,NaN,NaN
1,108,2023-01-02,-4.5,-7.4,804,-0.4,1525,NaN,NaN,NaN,...,0.6,2.7,6.2,9.9,15.8,17.9,1.3,1.9,NaN,NaN
2,108,2023-01-03,-5.0,-9.0,551,0.6,1555,NaN,NaN,NaN,...,0.5,2.6,6.1,9.8,15.7,17.9,1.3,1.9,NaN,NaN
3,108,2023-01-04,-1.8,-5.7,229,3.3,1510,NaN,NaN,NaN,...,0.3,2.5,6.0,9.7,15.6,17.9,1.5,2.1,NaN,NaN
4,108,2023-01-05,-1.6,-5.6,749,3.6,1536,NaN,NaN,NaN,...,0.2,2.4,5.9,9.5,15.5,17.8,0.9,1.2,NaN,NaN


In [ ]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 59 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   지점                   365 non-null    int64  
 1   일시                   365 non-null    object 
 2   평균기온(°C)             365 non-null    float64
 3   최저기온(°C)             365 non-null    float64
 4   최저기온 시각(hhmi)        365 non-null    int64  
 5   최고기온(°C)             365 non-null    float64
 6   최고기온 시각(hhmi)        365 non-null    int64  
 7   강수 계속시간(hr)          149 non-null    float64
 8   10분 최다 강수량(mm)       100 non-null    float64
 9   10분 최다강수량 시각(hhmi)   68 non-null     float64
 10  1시간 최다강수량(mm)        100 non-null    float64
 11  1시간 최다 강수량 시각(hhmi)  71 non-null     float64
 12  일강수량(mm)             150 non-null    float64
 13  최대 순간 풍속(m/s)        363 non-null    float64
 14  최대 순간 풍속 풍향(16방위)    363 non-null    float64
 15  최대 순간풍속 시각(hhmi)     363 non-null    flo

In [ ]:
weather_df.columns

Index(['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최저기온 시각(hhmi)', '최고기온(°C)',
       '최고기온 시각(hhmi)', '강수 계속시간(hr)', '10분 최다 강수량(mm)', '10분 최다강수량 시각(hhmi)',
       '1시간 최다강수량(mm)', '1시간 최다 강수량 시각(hhmi)', '일강수량(mm)', '최대 순간 풍속(m/s)',
       '최대 순간 풍속 풍향(16방위)', '최대 순간풍속 시각(hhmi)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)',
       '최대 풍속 시각(hhmi)', '평균 풍속(m/s)', '풍정합(100m)', '평균 이슬점온도(°C)',
       '최소 상대습도(%)', '최소 상대습도 시각(hhmi)', '평균 상대습도(%)', '평균 증기압(hPa)',
       '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최고 해면기압 시각(hhmi)', '최저 해면기압(hPa)',
       '최저 해면기압 시각(hhmi)', '평균 해면기압(hPa)', '가조시간(hr)', '합계 일조시간(hr)',
       '1시간 최다일사 시각(hhmi)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)', '일 최심신적설(cm)',
       '일 최심신적설 시각(hhmi)', '일 최심적설(cm)', '일 최심적설 시각(hhmi)', '합계 3시간 신적설(cm)',
       '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)', '최저 초상온도(°C)',
       '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)', '평균 20cm 지중온도(°C)',
       '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)', '1.0m 지중온도(°C)', '1.5m 지중온도(°C)',
       '3.0m 지중온도(°C)', '5.0m 지중온도(°C

### 1. 사용하지 않을 열 제거
- 시간과 관련된 hhmi 단위의 데이터들은 사용하지 않을 예정이기에 제거해줍니다
- 지점의 경우 항상 동일하게 제거해줍니다

In [ ]:
weather_df = weather_df[[column for column in weather_df.columns if column.rfind('(hhmi)')==-1 and column !='지점']]
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   일시                 365 non-null    object 
 1   평균기온(°C)           365 non-null    float64
 2   최저기온(°C)           365 non-null    float64
 3   최고기온(°C)           365 non-null    float64
 4   강수 계속시간(hr)        149 non-null    float64
 5   10분 최다 강수량(mm)     100 non-null    float64
 6   1시간 최다강수량(mm)      100 non-null    float64
 7   일강수량(mm)           150 non-null    float64
 8   최대 순간 풍속(m/s)      363 non-null    float64
 9   최대 순간 풍속 풍향(16방위)  363 non-null    float64
 10  최대 풍속(m/s)         363 non-null    float64
 11  최대 풍속 풍향(16방위)     363 non-null    float64
 12  평균 풍속(m/s)         363 non-null    float64
 13  풍정합(100m)          363 non-null    float64
 14  평균 이슬점온도(°C)       365 non-null    float64
 15  최소 상대습도(%)         365 non-null    int64  
 16  평균 상대습도(%)         365 non

In [ ]:
# 결측치 숫자
weather_df.isnull().sum().sort_values(ascending=False)

안개 계속시간(hr)          357
일 최심신적설(cm)          356
합계 3시간 신적설(cm)       356
일 최심적설(cm)           349
10분 최다 강수량(mm)       265
1시간 최다강수량(mm)        265
강수 계속시간(hr)          216
일강수량(mm)             215
9-9강수(mm)            210
평균 풍속(m/s)             2
합계 대형증발량(mm)           2
풍정합(100m)              2
최대 풍속 풍향(16방위)         2
최대 풍속(m/s)             2
최대 순간 풍속 풍향(16방위)      2
최대 순간 풍속(m/s)          2
합계 소형증발량(mm)           2
1시간 최다일사량(MJ/m2)       1
1.5m 지중온도(°C)          1
1.0m 지중온도(°C)          1
0.5m 지중온도(°C)          1
합계 일사량(MJ/m2)          1
합계 일조시간(hr)            1
평균기온(°C)               0
평균 20cm 지중온도(°C)       0
최저기온(°C)               0
최고기온(°C)               0
평균 이슬점온도(°C)           0
5.0m 지중온도(°C)          0
3.0m 지중온도(°C)          0
최소 상대습도(%)             0
평균 상대습도(%)             0
평균 증기압(hPa)            0
평균 30cm 지중온도(°C)       0
평균 10cm 지중온도(°C)       0
가조시간(hr)               0
평균 5cm 지중온도(°C)        0
최저 초상온도(°C)            0
평균 지면온도(°C)            0
평균 중하층운량(1/10)         0


### 2. 결측치 처리
- 위의 데이터를 보면 특정 열들은 결측치가 90%가 넘는 것으로 보인다. 안개 계속시간(hr), 일 최심신적설(cm) 등은 제거해줍니다.
- 강수량과 관련된 데이터의 경우 결측치일 경우 강수량이 0이라는 의미이기에 0으로 대체해줍니다.
- 날씨 관측데이터의 경우 연속성이 있기에 나머지 데이터는 보간해줍니다.

In [ ]:
# 결측치가 많은 열 삭제
weather_df = weather_df.drop(columns=['안개 계속시간(hr)', '일 최심신적설(cm)', '합계 3시간 신적설(cm)', '일 최심적설(cm)', '강수 계속시간(hr)', '1시간 최다강수량(mm)', '10분 최다 강수량(mm)'])
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   일시                 365 non-null    object 
 1   평균기온(°C)           365 non-null    float64
 2   최저기온(°C)           365 non-null    float64
 3   최고기온(°C)           365 non-null    float64
 4   일강수량(mm)           150 non-null    float64
 5   최대 순간 풍속(m/s)      363 non-null    float64
 6   최대 순간 풍속 풍향(16방위)  363 non-null    float64
 7   최대 풍속(m/s)         363 non-null    float64
 8   최대 풍속 풍향(16방위)     363 non-null    float64
 9   평균 풍속(m/s)         363 non-null    float64
 10  풍정합(100m)          363 non-null    float64
 11  평균 이슬점온도(°C)       365 non-null    float64
 12  최소 상대습도(%)         365 non-null    int64  
 13  평균 상대습도(%)         365 non-null    float64
 14  평균 증기압(hPa)        365 non-null    float64
 15  평균 현지기압(hPa)       365 non-null    float64
 16  최고 해면기압(hPa)       365 non

In [ ]:
# 결측치 숫자
weather_df.isnull().sum().sort_values(ascending=False)

일강수량(mm)             215
9-9강수(mm)            210
최대 순간 풍속 풍향(16방위)      2
풍정합(100m)              2
최대 풍속 풍향(16방위)         2
최대 풍속(m/s)             2
평균 풍속(m/s)             2
최대 순간 풍속(m/s)          2
합계 대형증발량(mm)           2
합계 소형증발량(mm)           2
합계 일사량(MJ/m2)          1
1시간 최다일사량(MJ/m2)       1
1.5m 지중온도(°C)          1
1.0m 지중온도(°C)          1
0.5m 지중온도(°C)          1
합계 일조시간(hr)            1
평균 10cm 지중온도(°C)       0
평균 5cm 지중온도(°C)        0
3.0m 지중온도(°C)          0
평균 20cm 지중온도(°C)       0
평균 30cm 지중온도(°C)       0
평균 지면온도(°C)            0
5.0m 지중온도(°C)          0
최저 초상온도(°C)            0
일시                     0
평균 중하층운량(1/10)         0
평균 전운량(1/10)           0
평균기온(°C)               0
평균 해면기압(hPa)           0
최저 해면기압(hPa)           0
최고 해면기압(hPa)           0
평균 현지기압(hPa)           0
평균 증기압(hPa)            0
평균 상대습도(%)             0
최소 상대습도(%)             0
평균 이슬점온도(°C)           0
최고기온(°C)               0
최저기온(°C)               0
가조시간(hr)               0
dtype: int64

In [ ]:
# 강수량 관련 데이터 처리
weather_df.columns

Index(['일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '일강수량(mm)', '최대 순간 풍속(m/s)',
       '최대 순간 풍속 풍향(16방위)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)', '평균 풍속(m/s)',
       '풍정합(100m)', '평균 이슬점온도(°C)', '최소 상대습도(%)', '평균 상대습도(%)', '평균 증기압(hPa)',
       '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최저 해면기압(hPa)', '평균 해면기압(hPa)',
       '가조시간(hr)', '합계 일조시간(hr)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)',
       '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)', '최저 초상온도(°C)',
       '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)', '평균 20cm 지중온도(°C)',
       '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)', '1.0m 지중온도(°C)', '1.5m 지중온도(°C)',
       '3.0m 지중온도(°C)', '5.0m 지중온도(°C)', '합계 대형증발량(mm)', '합계 소형증발량(mm)',
       '9-9강수(mm)'],
      dtype='object')

In [ ]:
# 강수량 관련 열들: '9-9강수(mm)', '일강수량(mm)'

for column in ['9-9강수(mm)', '일강수량(mm)']:
    weather_df[column].fillna(0, inplace=True)

weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   일시                 365 non-null    object 
 1   평균기온(°C)           365 non-null    float64
 2   최저기온(°C)           365 non-null    float64
 3   최고기온(°C)           365 non-null    float64
 4   일강수량(mm)           365 non-null    float64
 5   최대 순간 풍속(m/s)      363 non-null    float64
 6   최대 순간 풍속 풍향(16방위)  363 non-null    float64
 7   최대 풍속(m/s)         363 non-null    float64
 8   최대 풍속 풍향(16방위)     363 non-null    float64
 9   평균 풍속(m/s)         363 non-null    float64
 10  풍정합(100m)          363 non-null    float64
 11  평균 이슬점온도(°C)       365 non-null    float64
 12  최소 상대습도(%)         365 non-null    int64  
 13  평균 상대습도(%)         365 non-null    float64
 14  평균 증기압(hPa)        365 non-null    float64
 15  평균 현지기압(hPa)       365 non-null    float64
 16  최고 해면기압(hPa)       365 non

In [ ]:
# 결측치 숫자
weather_df.isnull().sum().sort_values(ascending=False)

최대 풍속 풍향(16방위)       2
합계 소형증발량(mm)         2
합계 대형증발량(mm)         2
최대 순간 풍속(m/s)        2
최대 순간 풍속 풍향(16방위)    2
최대 풍속(m/s)           2
평균 풍속(m/s)           2
풍정합(100m)            2
합계 일조시간(hr)          1
0.5m 지중온도(°C)        1
1.0m 지중온도(°C)        1
1.5m 지중온도(°C)        1
1시간 최다일사량(MJ/m2)     1
합계 일사량(MJ/m2)        1
평균 10cm 지중온도(°C)     0
최저 초상온도(°C)          0
평균 5cm 지중온도(°C)      0
일시                   0
평균 20cm 지중온도(°C)     0
평균 30cm 지중온도(°C)     0
평균 중하층운량(1/10)       0
3.0m 지중온도(°C)        0
5.0m 지중온도(°C)        0
평균 지면온도(°C)          0
가조시간(hr)             0
평균 전운량(1/10)         0
평균기온(°C)             0
평균 해면기압(hPa)         0
최저 해면기압(hPa)         0
최고 해면기압(hPa)         0
평균 현지기압(hPa)         0
평균 증기압(hPa)          0
평균 상대습도(%)           0
최소 상대습도(%)           0
평균 이슬점온도(°C)         0
일강수량(mm)             0
최고기온(°C)             0
최저기온(°C)             0
9-9강수(mm)            0
dtype: int64

In [ ]:
# 남은 결측치 보간
weather_df = weather_df.interpolate(method='linear')
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   일시                 365 non-null    object 
 1   평균기온(°C)           365 non-null    float64
 2   최저기온(°C)           365 non-null    float64
 3   최고기온(°C)           365 non-null    float64
 4   일강수량(mm)           365 non-null    float64
 5   최대 순간 풍속(m/s)      365 non-null    float64
 6   최대 순간 풍속 풍향(16방위)  365 non-null    float64
 7   최대 풍속(m/s)         365 non-null    float64
 8   최대 풍속 풍향(16방위)     365 non-null    float64
 9   평균 풍속(m/s)         365 non-null    float64
 10  풍정합(100m)          365 non-null    float64
 11  평균 이슬점온도(°C)       365 non-null    float64
 12  최소 상대습도(%)         365 non-null    int64  
 13  평균 상대습도(%)         365 non-null    float64
 14  평균 증기압(hPa)        365 non-null    float64
 15  평균 현지기압(hPa)       365 non-null    float64
 16  최고 해면기압(hPa)       365 non

In [ ]:
# 결측치 숫자
weather_df.isnull().sum().sort_values(ascending=False)

일시                   0
평균 20cm 지중온도(°C)     0
합계 일사량(MJ/m2)        0
평균 전운량(1/10)         0
평균 중하층운량(1/10)       0
평균 지면온도(°C)          0
최저 초상온도(°C)          0
평균 5cm 지중온도(°C)      0
평균 10cm 지중온도(°C)     0
평균 30cm 지중온도(°C)     0
합계 일조시간(hr)          0
0.5m 지중온도(°C)        0
1.0m 지중온도(°C)        0
1.5m 지중온도(°C)        0
3.0m 지중온도(°C)        0
5.0m 지중온도(°C)        0
합계 대형증발량(mm)         0
합계 소형증발량(mm)         0
1시간 최다일사량(MJ/m2)     0
가조시간(hr)             0
평균기온(°C)             0
평균 풍속(m/s)           0
최저기온(°C)             0
최고기온(°C)             0
일강수량(mm)             0
최대 순간 풍속(m/s)        0
최대 순간 풍속 풍향(16방위)    0
최대 풍속(m/s)           0
최대 풍속 풍향(16방위)       0
풍정합(100m)            0
평균 해면기압(hPa)         0
평균 이슬점온도(°C)         0
최소 상대습도(%)           0
평균 상대습도(%)           0
평균 증기압(hPa)          0
평균 현지기압(hPa)         0
최고 해면기압(hPa)         0
최저 해면기압(hPa)         0
9-9강수(mm)            0
dtype: int64

### 3. 날씨 데이터 합치기
- 날씨 데이터의 경우 1년씩 총 3개의 데이터가 있습니다. 모든 데이터에 1-2번을 진행해준 후 하나의 데이터베이스로 합쳐야합니다

In [ ]:
# 위의 데이터 처리를 해주는 함수 생성
def data_preprocess_weather(file_path):
    df = pd.read_csv(file_path, encoding='cp949')
    df = df[[column for column in df.columns if column.rfind('(hhmi)')==-1 and column !='지점']]
    df = df.drop(columns=['안개 계속시간(hr)', '일 최심신적설(cm)', '합계 3시간 신적설(cm)', '일 최심적설(cm)', '강수 계속시간(hr)', '1시간 최다강수량(mm)', '10분 최다 강수량(mm)'])
    for column in ['9-9강수(mm)', '일강수량(mm)']:
        df[column].fillna(0, inplace=True)
    df = df.interpolate(method='linear')
    return df

In [ ]:
os.listdir('data/원본/날씨데이터/일별')

['SURFACE_ASOS_108_DAY_2022_2022_2023.csv',
 'SURFACE_ASOS_108_DAY_2023_2023_2024.csv',
 'SURFACE_ASOS_108_DAY_2024_2024_2025.csv']

In [ ]:
df_list = []
for file in os.listdir('data/원본/날씨데이터/일별'):
    file_path = 'data/원본/날씨데이터/일별/' + file
    df_list.append(data_preprocess_weather(file_path))
weather_df = pd.concat(df_list)
weather_df.shape

(1096, 39)

In [ ]:
# 결측치 여부
weather_df.isnull().sum().sort_values(ascending=False)

일시                   0
평균 20cm 지중온도(°C)     0
합계 일사량(MJ/m2)        0
평균 전운량(1/10)         0
평균 중하층운량(1/10)       0
평균 지면온도(°C)          0
최저 초상온도(°C)          0
평균 5cm 지중온도(°C)      0
평균 10cm 지중온도(°C)     0
평균 30cm 지중온도(°C)     0
합계 일조시간(hr)          0
0.5m 지중온도(°C)        0
1.0m 지중온도(°C)        0
1.5m 지중온도(°C)        0
3.0m 지중온도(°C)        0
5.0m 지중온도(°C)        0
합계 대형증발량(mm)         0
합계 소형증발량(mm)         0
1시간 최다일사량(MJ/m2)     0
가조시간(hr)             0
평균기온(°C)             0
평균 풍속(m/s)           0
최저기온(°C)             0
최고기온(°C)             0
일강수량(mm)             0
최대 순간 풍속(m/s)        0
최대 순간 풍속 풍향(16방위)    0
최대 풍속(m/s)           0
최대 풍속 풍향(16방위)       0
풍정합(100m)            0
평균 해면기압(hPa)         0
평균 이슬점온도(°C)         0
최소 상대습도(%)           0
평균 상대습도(%)           0
평균 증기압(hPa)          0
평균 현지기압(hPa)         0
최고 해면기압(hPa)         0
최저 해면기압(hPa)         0
9-9강수(mm)            0
dtype: int64

In [ ]:
weather_df.to_csv('data/전처리/날씨데이터_2022_2024.csv', index=False)

## 전력데이터 + 날씨데이터
- 마지막으로 두 데이터를 합쳐 하나의 데이터로 만들어줍니다

In [1]:
import pandas as pd
import numpy as np

In [2]:
weather_df = pd.read_csv('data/전처리/날씨데이터_2022_2024.csv', )
weather_df.tail()

,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),최대 순간 풍속 풍향(16방위),최대 풍속(m/s),최대 풍속 풍향(16방위),평균 풍속(m/s),...,평균 20cm 지중온도(°C),평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm)
1091,2024-12-27,-2.6,-5.9,1.8,0.0,10.4,290.0,5.8,320.0,2.7,...,1.0,2.0,3.7,7.5,10.7,17.3,18.2,1.3,1.8,0.0
1092,2024-12-28,-3.0,-6.7,1.1,0.0,9.9,290.0,5.8,290.0,2.5,...,0.9,1.9,3.6,7.4,10.5,17.2,18.2,1.3,1.9,0.0
1093,2024-12-29,1.1,-4.2,6.1,0.0,5.0,180.0,2.8,250.0,1.5,...,0.7,1.7,3.5,7.2,10.4,17.0,18.2,1.3,1.8,0.0
1094,2024-12-30,5.5,1.9,10.1,0.0,12.0,230.0,6.9,230.0,2.3,...,0.7,1.7,3.4,7.1,10.3,16.9,18.1,1.4,2.0,0.0
1095,2024-12-31,0.6,-2.0,4.9,0.0,9.6,290.0,5.0,270.0,2.6,...,0.8,1.7,3.4,7.0,10.1,16.8,18.1,1.6,2.4,0.0


In [3]:
power_df = pd.read_csv('data/전처리/법정동_일별_전력량.csv')
power_df.tail()

,전체코드,시군구코드,시군구명,법정동코드,법정동명,날짜,파워
181403,1174011000,11740,강동구,11000,강일동,2024-01-24,115124.547
181404,1174011000,11740,강동구,11000,강일동,2024-01-25,99907.500
181405,1174011000,11740,강동구,11000,강일동,2024-01-26,94818.512
181406,1174011000,11740,강동구,11000,강일동,2024-01-27,86792.828
181407,1174011000,11740,강동구,11000,강일동,2024-01-28,86389.477


In [7]:
weather_df.shape, power_df.shape

((1096, 39), (181408, 7))

In [ ]:
merged = power_df.merge(weather_df, how='left', left_on='날짜', right_on='일시')
merged = merged.drop(columns=['일시'])
merged.shape

(181408, 45)

In [10]:
merged.head()

,전체코드,시군구코드,시군구명,법정동코드,법정동명,날짜,파워,평균기온(°C),최저기온(°C),최고기온(°C),...,평균 20cm 지중온도(°C),평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm)
0,1111010100,11110,종로구,10100,청운동,2022-12-04,146294.6135,-3.5,-5.2,-0.2,...,3.5,5.2,7.5,12.1,15.6,18.4,18.7,1.2,1.7,0.0
1,1111010100,11110,종로구,10100,청운동,2022-12-05,175633.8270,-3.1,-7.0,1.9,...,2.8,4.6,7.0,11.7,15.3,18.4,18.7,1.5,2.1,0.1
2,1111010100,11110,종로구,10100,청운동,2022-12-06,156084.1910,0.9,-3.9,5.3,...,2.4,4.1,6.6,11.3,15.1,18.3,18.7,1.3,1.9,0.1
3,1111010100,11110,종로구,10100,청운동,2022-12-08,177018.8420,3.7,0.1,9.1,...,3.6,4.8,6.6,10.7,14.5,18.2,18.6,1.5,2.2,0.0
4,1111010100,11110,종로구,10100,청운동,2022-12-09,158467.1690,4.5,0.9,10.2,...,3.9,4.9,6.6,10.5,14.3,18.1,18.6,1.2,1.8,0.0


In [13]:
merged.isnull().sum().sort_values(ascending=False)

전체코드                 0
최저 해면기압(hPa)         0
가조시간(hr)             0
합계 일조시간(hr)          0
1시간 최다일사량(MJ/m2)     0
합계 일사량(MJ/m2)        0
평균 전운량(1/10)         0
평균 중하층운량(1/10)       0
평균 지면온도(°C)          0
최저 초상온도(°C)          0
평균 5cm 지중온도(°C)      0
평균 10cm 지중온도(°C)     0
평균 20cm 지중온도(°C)     0
평균 30cm 지중온도(°C)     0
0.5m 지중온도(°C)        0
1.0m 지중온도(°C)        0
1.5m 지중온도(°C)        0
3.0m 지중온도(°C)        0
5.0m 지중온도(°C)        0
합계 대형증발량(mm)         0
합계 소형증발량(mm)         0
평균 해면기압(hPa)         0
최고 해면기압(hPa)         0
시군구코드                0
평균 현지기압(hPa)         0
시군구명                 0
법정동코드                0
법정동명                 0
날짜                   0
파워                   0
평균기온(°C)             0
최저기온(°C)             0
최고기온(°C)             0
일강수량(mm)             0
최대 순간 풍속(m/s)        0
최대 순간 풍속 풍향(16방위)    0
최대 풍속(m/s)           0
최대 풍속 풍향(16방위)       0
평균 풍속(m/s)           0
풍정합(100m)            0
평균 이슬점온도(°C)         0
최소 상대습도(%)           0
평균 상대습도(%)           0
평균 증기압(hPa)

In [11]:
merged.to_csv('data/전처리/법정동_일별_날씨_전력량.csv', index=False)